### Train MNIST via Keras

- https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [1]:
from __future__ import print_function

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


- Run the cell below if "failed to create cublas handle: CUBLAS_STATUS_ALLOC_FAILED" error happens...

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 14s 228us/step - loss: 0.2706 - acc: 0.9169 - val_loss: 0.0522 - val_acc: 0.9822
Epoch 2/12
60000/60000 [==============================] - 5s 77us/step - loss: 0.0859 - acc: 0.9745 - val_loss: 0.0456 - val_acc: 0.9850
Epoch 3/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0661 - acc: 0.9801 - val_loss: 0.0399 - val_acc: 0.9872
Epoch 4/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0525 - acc: 0.9842 - val_loss: 0.0304 - val_acc: 0.9900
Epoch 5/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0463 - acc: 0.9857 - val_loss: 0.0286 - val_acc: 0.9899
Epoch 6/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0420 - acc: 0.9874 - val_loss: 0.0283 - val_acc: 0.9909
Epoch 7/12
60000/60000 [==============================] - 5s 80us/step - loss: 0.0368 - acc: 0.9886 - val_loss: 0.0305 - val_a

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.027889487889770862
Test accuracy: 0.9917


#### Save model

In [ ]:
model.save('lenet.h5')